In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
from keras.layers import LSTM, Dense, SimpleRNN
import matplotlib.pyplot as plt
import statistics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [5]:
import glob
files = glob.glob("../../datasets/prop-4.csv", recursive = True)
ant_master = pd.concat(map(pd.read_csv, files))

In [6]:
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...  dam  moa  \
0     prop-4      285    2    3    0   16   16     1   0  16  ...  0.0    0   
1     prop-4      285    2    4    0   23   19     1   0  23  ...  0.0    0   
2     prop-4      285    4    6    0    8   13     4   0   8  ...  0.0    0   
3     prop-4      285    4    5    2   23   53     4   2  21  ...  0.0    0   
4     prop-4      285    2    2    0   11   10     1   0  11  ...  0.0    0   
...      ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...  ...  ...   
8713  prop-4      318    0    1    0    0    0     0   0   0  ...  0.0    0   
8714  prop-4      318    0    1    0    0    0     0   0   0  ...  0.0    0   
8715  prop-4      318   20    2    0   10   63   140   1   9  ...  0.8    6   
8716  prop-4      318   12    2    0    8   30    66   1   7  ...  0.0    0   
8717  prop-4      318    0    1    0    0    0     0   0   0  ...  0.0    0   

           mfa       cam  ic  cbm    amc  max_cc  avg_cc  bug  
0     0.900000  0.625000   1    1  62.50       1  0.5000    0  
1     0.933333  0.600000   1    1  34.50       1  0.5000    0  
2     0.913043  0.555556   2    2   8.75       1  0.5000    0  
3     0.960784  0.555556   1    2  70.75       1  0.5000    0  
4     0.800000  0.666667   1    1  18.50       1  0.5000    0  
...        ...       ...  ..  ...    ...     ...     ...  ...  
8713  0.000000  0.000000   0    0   0.00       0  0.0000    0  
8714  0.000000  0.000000   0    0   0.00       0  0.0000    0  
8715  0.500000  0.302632   0    0  17.20       5  1.3500    0  
8716  0.620690  0.388889   0    0  21.25       6  1.9167    0  
8717  0.000000  0.000000   0    0   0.00       0  0.0000    0  

[8718 rows x 23 columns]

In [7]:
ant_master = ant_master.loc[ant_master['version'] == 285]
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...  dam  moa  \
0     prop-4      285    2    3    0   16   16     1   0  16  ...  0.0    0   
1     prop-4      285    2    4    0   23   19     1   0  23  ...  0.0    0   
2     prop-4      285    4    6    0    8   13     4   0   8  ...  0.0    0   
3     prop-4      285    4    5    2   23   53     4   2  21  ...  0.0    0   
4     prop-4      285    2    2    0   11   10     1   0  11  ...  0.0    0   
...      ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...  ...  ...   
1689  prop-4      285    1    1    0    1    5     0   0   1  ...  0.0    0   
1690  prop-4      285    1    1    0    0    5     0   0   0  ...  0.0    0   
1691  prop-4      285    1    1    0    2    5     0   1   1  ...  0.0    0   
1692  prop-4      285    1    1    0    1    5     0   0   1  ...  0.0    0   
1693  prop-4      285    1    1    0    0    5     0   0   0  ...  0.0    0   

           mfa       cam  ic  cbm    amc  max_cc  avg_cc  bug  
0     0.900000  0.625000   1    1  62.50       1     0.5    0  
1     0.933333  0.600000   1    1  34.50       1     0.5    0  
2     0.913043  0.555556   2    2   8.75       1     0.5    0  
3     0.960784  0.555556   1    2  70.75       1     0.5    0  
4     0.800000  0.666667   1    1  18.50       1     0.5    0  
...        ...       ...  ..  ...    ...     ...     ...  ...  
1689  0.000000  0.000000   0    0  12.00       0     0.0    0  
1690  0.000000  0.000000   0    0  12.00       0     0.0    0  
1691  0.000000  0.000000   0    0  12.00       0     0.0    0  
1692  0.000000  0.000000   0    0  12.00       0     0.0    0  
1693  0.000000  0.000000   0    0  12.00       0     0.0    0  

[1694 rows x 23 columns]

In [8]:
train, test = train_test_split(ant_master, test_size=0.2)

In [9]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [10]:
x_train

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  loc       dam  \
1010    5    6    0    8   14     8   0   8    1  1.000000   45  0.000000   
271     5    5    0    9   11    10   1   8    2  1.083333   42  0.000000   
1121    2    3    0   23   26     1   0  23    2  1.000000   99  0.000000   
676     3    3    0    6    7     3   0   6    1  1.500000   30  1.000000   
960     2    3    0    8   10     1   0   8    2  2.000000   37  0.000000   
...   ...  ...  ...  ...  ...   ...  ..  ..  ...       ...  ...       ...   
1066    5    5    0   11   12    10   0  11    1  2.000000   34  0.000000   
1135    2    3    0   16   13     1   0  16    2  1.000000   46  0.000000   
863     7    3    0   28   60    17   0  28    6  0.969697  325  0.181818   
179     4    5    0   14   40     4   1  13    3  1.000000  174  0.000000   
835     6    6    0   20   26     3   0  20    1  0.866667   99  0.000000   

      moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  
1010    0  0.875000  0.500000   3    3   7.400000       1  0.6000  
271     0  0.914286  0.625000   3    5   6.800000       1  0.6000  
1121    0  0.857143  0.625000   1    1  48.000000       1  0.5000  
676     0  0.916667  0.777778   1    2   8.333333       1  0.6667  
960     0  0.857143  0.625000   1    1  17.500000       1  0.5000  
...   ...       ...       ...  ..  ...        ...     ...     ...  
1066    0  0.885714  0.700000   3    4   5.800000       2  1.0000  
1135    0  0.857143  0.625000   1    1  21.500000       1  0.5000  
863     0  0.821429  0.500000   0    0  43.857143       8  2.5714  
179     0  0.960784  0.555556   4    8  41.750000       1  0.5000  
835     0  0.840000  0.733333   2    3  15.000000       1  0.6667  

[1355 rows x 20 columns]

In [11]:
x_test

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  loc       dam  \
1358    1    1    0    0    5     0   0   0    0  2.000000   18  0.000000   
1667    2    1    0    2    5     1   0   2    1  2.000000   13  0.000000   
1380   14    1    0   28   74    87   1  27    9  0.923077  890  0.333333   
394     2    3    0   16   19     1   0  16    2  2.000000   90  0.000000   
734     4    5    0   13   31     4   0  13    3  1.000000  196  0.000000   
...   ...  ...  ...  ...  ...   ...  ..  ..  ...       ...  ...       ...   
1600    1    1    0    1    5     0   0   1    0  2.000000   38  0.000000   
1593    1    1    0    0    5     0   0   0    0  2.000000   18  0.000000   
1678    1    1    0    1    5     0   0   1    0  2.000000   16  0.000000   
602     4    5    1   19   29     6   1  18    2  1.111111   96  0.000000   
1198    2    3    0   18   16     1   0  18    2  2.000000  251  0.000000   

      moa       mfa       cam  ic  cbm         amc  max_cc  avg_cc  
1358    0  0.000000  0.000000   0    0   12.000000       0  0.0000  
1667    0  0.000000  0.500000   0    0    5.500000       1  0.5000  
1380    0  0.000000  0.128205   0    0   62.357143      11  2.8571  
394     0  0.857143  0.625000   1    1   44.000000       1  0.5000  
734     0  0.960784  0.555556   4    9   47.250000       1  0.5000  
...   ...       ...       ...  ..  ...         ...     ...     ...  
1600    0  0.000000  0.000000   0    0   12.000000       0  0.0000  
1593    0  0.000000  0.000000   0    0   12.000000       0  0.0000  
1678    0  0.000000  0.000000   0    0   12.000000       0  0.0000  
602     0  0.944444  0.555556   2    2   22.250000       1  0.5000  
1198    0  0.857143  0.625000   1    1  124.500000       1  0.5000  

[339 rows x 20 columns]

In [12]:
y_train = train['bug']
y_test = test['bug']

In [13]:
y_train

1010    0
271     0
1121    0
676     0
960     0
       ..
1066    0
1135    0
863     0
179     0
835     0
Name: bug, Length: 1355, dtype: int64

In [14]:
y_test

1358    0
1667    0
1380    0
394     0
734     0
       ..
1600    0
1593    0
1678    0
602     0
1198    1
Name: bug, Length: 339, dtype: int64

In [15]:
x_train_np = np.array(x_train)
x_test_np = np.array(x_test)

In [16]:
x_train_np

array([[ 5.        ,  6.        ,  0.        , ...,  7.4       ,
         1.        ,  0.6       ],
       [ 5.        ,  5.        ,  0.        , ...,  6.8       ,
         1.        ,  0.6       ],
       [ 2.        ,  3.        ,  0.        , ..., 48.        ,
         1.        ,  0.5       ],
       ...,
       [ 7.        ,  3.        ,  0.        , ..., 43.85714286,
         8.        ,  2.5714    ],
       [ 4.        ,  5.        ,  0.        , ..., 41.75      ,
         1.        ,  0.5       ],
       [ 6.        ,  6.        ,  0.        , ..., 15.        ,
         1.        ,  0.6667    ]])

In [17]:
x_test_np

array([[  1.        ,   1.        ,   0.        , ...,  12.        ,
          0.        ,   0.        ],
       [  2.        ,   1.        ,   0.        , ...,   5.5       ,
          1.        ,   0.5       ],
       [ 14.        ,   1.        ,   0.        , ...,  62.35714286,
         11.        ,   2.8571    ],
       ...,
       [  1.        ,   1.        ,   0.        , ...,  12.        ,
          0.        ,   0.        ],
       [  4.        ,   5.        ,   1.        , ...,  22.25      ,
          1.        ,   0.5       ],
       [  2.        ,   3.        ,   0.        , ..., 124.5       ,
          1.        ,   0.5       ]])

In [18]:
train_x = np.reshape(x_train_np, (x_train_np.shape[0], 1, x_train_np.shape[1]))
test_x = np.reshape(x_test_np, (x_test_np.shape[0], 1, x_test_np.shape[1]))

In [19]:
train_y = y_train.to_numpy()
test_y = y_test.to_numpy()

In [20]:
train_x

array([[[ 5.        ,  6.        ,  0.        , ...,  7.4       ,
          1.        ,  0.6       ]],

       [[ 5.        ,  5.        ,  0.        , ...,  6.8       ,
          1.        ,  0.6       ]],

       [[ 2.        ,  3.        ,  0.        , ..., 48.        ,
          1.        ,  0.5       ]],

       ...,

       [[ 7.        ,  3.        ,  0.        , ..., 43.85714286,
          8.        ,  2.5714    ]],

       [[ 4.        ,  5.        ,  0.        , ..., 41.75      ,
          1.        ,  0.5       ]],

       [[ 6.        ,  6.        ,  0.        , ..., 15.        ,
          1.        ,  0.6667    ]]])

In [21]:
test_y

array([0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 2, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7,
       0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       1, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [22]:
def define_model():
    # Designing and initializing the model.
    model = Sequential()
    model.add(LSTM(200, return_sequences=True))
    model.add(LSTM(100, return_sequences=True))
    model.add(LSTM(50, dropout = 0.2, return_sequences=True))
    model.add(LSTM(20, return_sequences=False))
    model.add(Dense(1, activation = 'relu'))
    model.compile(loss = 'mse' , optimizer = keras.optimizers.Adam(learning_rate=0.0005) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )
    return model

In [23]:
y = list(test_y)

In [24]:
total_training_time = []
total_scores = []
total_training_loss_history = []
test_ARE = []
for i in range(10):
    print("Run no: " + str(i))
    model = define_model()
    log_dir = '/home/bavanya/Documents/Training_curves/propv285/' + 'run' + str(i)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    start = time.time()
    history = model.fit(train_x, train_y, epochs = 100, batch_size = 100, callbacks=[tensorboard_callback])
    stop = time.time()
    total_training_time.append(stop - start) 
    total_training_loss_history.append(history.history)
    score = model.evaluate(test_x, test_y)
    total_scores.append(score) 
    predictions = model.predict(test_x)
    x = predictions.tolist()
    x = list(np.concatenate(x))
    x = [int(i) for i in x]
    ARE_test = tf.keras.metrics.MeanRelativeError(normalizer=y)
    ARE_test.update_state(y, x)
    test_ARE.append(ARE_test.result().numpy())
    print("score of " + str(i) + "th run is: " + ', '.join(map(str,score)))

Run no: 0
Epoch 1/100
14/14 [==============================] - 8s 36ms/step - loss: 0.2640 - mse: 0.2640 - mae: 0.1681 - root_mean_squared_error: 0.5138 - mean_squared_logarithmic_error: 0.0760
Epoch 2/100
14/14 [==============================] - 0s 20ms/step - loss: 0.2495 - mse: 0.2495 - mae: 0.2319 - root_mean_squared_error: 0.4995 - mean_squared_logarithmic_error: 0.0722
Epoch 3/100
14/14 [==============================] - 0s 21ms/step - loss: 0.2434 - mse: 0.2434 - mae: 0.2544 - root_mean_squared_error: 0.4933 - mean_squared_logarithmic_error: 0.0722
Epoch 4/100
14/14 [==============================] - 0s 21ms/step - loss: 0.2370 - mse: 0.2370 - mae: 0.2351 - root_mean_squared_error: 0.4868 - mean_squared_logarithmic_error: 0.0692
Epoch 5/100
14/14 [==============================] - 0s 21ms/step - loss: 0.2292 - mse: 0.2292 - mae: 0.2313 - root_mean_squared_error: 0.4787 - mean_squared_logarithmic_error: 0.0682
Epoch 6/100
14/14 [==============================] - 0s 21ms/step - lo

14/14 [==============================] - 0s 20ms/step - loss: 0.0986 - mse: 0.0986 - mae: 0.1310 - root_mean_squared_error: 0.3140 - mean_squared_logarithmic_error: 0.0392
Epoch 89/100
14/14 [==============================] - 0s 21ms/step - loss: 0.0994 - mse: 0.0994 - mae: 0.1356 - root_mean_squared_error: 0.3153 - mean_squared_logarithmic_error: 0.0380
Epoch 90/100
14/14 [==============================] - 0s 21ms/step - loss: 0.0880 - mse: 0.0880 - mae: 0.1282 - root_mean_squared_error: 0.2966 - mean_squared_logarithmic_error: 0.0354
Epoch 91/100
14/14 [==============================] - 0s 20ms/step - loss: 0.1033 - mse: 0.1033 - mae: 0.1416 - root_mean_squared_error: 0.3214 - mean_squared_logarithmic_error: 0.0404
Epoch 92/100
14/14 [==============================] - 0s 21ms/step - loss: 0.1094 - mse: 0.1094 - mae: 0.1321 - root_mean_squared_error: 0.3308 - mean_squared_logarithmic_error: 0.0405
Epoch 93/100
14/14 [==============================] - 0s 21ms/step - loss: 0.1147 - mse:

14/14 [==============================] - 0s 22ms/step - loss: 0.1199 - mse: 0.1199 - mae: 0.1427 - root_mean_squared_error: 0.3463 - mean_squared_logarithmic_error: 0.0431
Epoch 74/100
14/14 [==============================] - 0s 21ms/step - loss: 0.1095 - mse: 0.1095 - mae: 0.1489 - root_mean_squared_error: 0.3310 - mean_squared_logarithmic_error: 0.0420
Epoch 75/100
14/14 [==============================] - 0s 20ms/step - loss: 0.1097 - mse: 0.1097 - mae: 0.1394 - root_mean_squared_error: 0.3312 - mean_squared_logarithmic_error: 0.0399
Epoch 76/100
14/14 [==============================] - 0s 21ms/step - loss: 0.1094 - mse: 0.1094 - mae: 0.1490 - root_mean_squared_error: 0.3308 - mean_squared_logarithmic_error: 0.0411
Epoch 77/100
14/14 [==============================] - 0s 21ms/step - loss: 0.1070 - mse: 0.1070 - mae: 0.1411 - root_mean_squared_error: 0.3271 - mean_squared_logarithmic_error: 0.0394
Epoch 78/100
14/14 [==============================] - 0s 21ms/step - loss: 0.1140 - mse:

14/14 [==============================] - 0s 23ms/step - loss: 0.1134 - mse: 0.1134 - mae: 0.1609 - root_mean_squared_error: 0.3368 - mean_squared_logarithmic_error: 0.0441
Epoch 59/100
14/14 [==============================] - 0s 22ms/step - loss: 0.1128 - mse: 0.1128 - mae: 0.1578 - root_mean_squared_error: 0.3358 - mean_squared_logarithmic_error: 0.0435
Epoch 60/100
14/14 [==============================] - 0s 22ms/step - loss: 0.1085 - mse: 0.1085 - mae: 0.1321 - root_mean_squared_error: 0.3293 - mean_squared_logarithmic_error: 0.0409
Epoch 61/100
14/14 [==============================] - 0s 22ms/step - loss: 0.1077 - mse: 0.1077 - mae: 0.1579 - root_mean_squared_error: 0.3282 - mean_squared_logarithmic_error: 0.0419
Epoch 62/100
14/14 [==============================] - 0s 22ms/step - loss: 0.1147 - mse: 0.1147 - mae: 0.1576 - root_mean_squared_error: 0.3386 - mean_squared_logarithmic_error: 0.0430
Epoch 63/100
14/14 [==============================] - 0s 21ms/step - loss: 0.1143 - mse:

Epoch 1/100
14/14 [==============================] - 9s 64ms/step - loss: 0.2666 - mse: 0.2666 - mae: 0.1570 - root_mean_squared_error: 0.5163 - mean_squared_logarithmic_error: 0.0773
Epoch 2/100
14/14 [==============================] - 0s 20ms/step - loss: 0.2506 - mse: 0.2506 - mae: 0.2165 - root_mean_squared_error: 0.5006 - mean_squared_logarithmic_error: 0.0717
Epoch 3/100
14/14 [==============================] - 0s 23ms/step - loss: 0.2427 - mse: 0.2427 - mae: 0.2656 - root_mean_squared_error: 0.4927 - mean_squared_logarithmic_error: 0.0732
Epoch 4/100
14/14 [==============================] - 0s 22ms/step - loss: 0.2371 - mse: 0.2371 - mae: 0.2319 - root_mean_squared_error: 0.4869 - mean_squared_logarithmic_error: 0.0689
Epoch 5/100
14/14 [==============================] - 0s 19ms/step - loss: 0.2288 - mse: 0.2288 - mae: 0.2263 - root_mean_squared_error: 0.4783 - mean_squared_logarithmic_error: 0.0669
Epoch 6/100
14/14 [==============================] - 0s 23ms/step - loss: 0.2218

...

14/14 [==============================] - 0s 20ms/step - loss: 0.1034 - mse: 0.1034 - mae: 0.1479 - root_mean_squared_error: 0.3216 - mean_squared_logarithmic_error: 0.0405
Epoch 90/100
14/14 [==============================] - 0s 22ms/step - loss: 0.1054 - mse: 0.1054 - mae: 0.1190 - root_mean_squared_error: 0.3246 - mean_squared_logarithmic_error: 0.0389
Epoch 91/100
14/14 [==============================] - 0s 21ms/step - loss: 0.0956 - mse: 0.0956 - mae: 0.1271 - root_mean_squared_error: 0.3091 - mean_squared_logarithmic_error: 0.0377
Epoch 92/100
14/14 [==============================] - 0s 21ms/step - loss: 0.0950 - mse: 0.0950 - mae: 0.1357 - root_mean_squared_error: 0.3082 - mean_squared_logarithmic_error: 0.0378
Epoch 93/100
14/14 [==============================] - 0s 21ms/step - loss: 0.0917 - mse: 0.0917 - mae: 0.1315 - root_mean_squared_error: 0.3027 - mean_squared_logarithmic_error: 0.0358
Epoch 94/100
14/14 [==============================] - 0s 21ms/step - loss: 0.0916 - mse:

14/14 [==============================] - 0s 21ms/step - loss: 0.1002 - mse: 0.1002 - mae: 0.1320 - root_mean_squared_error: 0.3166 - mean_squared_logarithmic_error: 0.0391
Epoch 75/100
14/14 [==============================] - 0s 21ms/step - loss: 0.0959 - mse: 0.0959 - mae: 0.1382 - root_mean_squared_error: 0.3097 - mean_squared_logarithmic_error: 0.0380
Epoch 76/100
14/14 [==============================] - 0s 20ms/step - loss: 0.1074 - mse: 0.1074 - mae: 0.1404 - root_mean_squared_error: 0.3277 - mean_squared_logarithmic_error: 0.0390
Epoch 77/100
14/14 [==============================] - 0s 21ms/step - loss: 0.1170 - mse: 0.1170 - mae: 0.1310 - root_mean_squared_error: 0.3420 - mean_squared_logarithmic_error: 0.0411
Epoch 78/100
14/14 [==============================] - 0s 21ms/step - loss: 0.0993 - mse: 0.0993 - mae: 0.1358 - root_mean_squared_error: 0.3151 - mean_squared_logarithmic_error: 0.0385
Epoch 79/100
14/14 [==============================] - 0s 21ms/step - loss: 0.1031 - mse:

14/14 [==============================] - 0s 20ms/step - loss: 0.1234 - mse: 0.1234 - mae: 0.1601 - root_mean_squared_error: 0.3512 - mean_squared_logarithmic_error: 0.0445
Epoch 60/100
14/14 [==============================] - 0s 21ms/step - loss: 0.1147 - mse: 0.1147 - mae: 0.1549 - root_mean_squared_error: 0.3387 - mean_squared_logarithmic_error: 0.0421
Epoch 61/100
14/14 [==============================] - 0s 21ms/step - loss: 0.1099 - mse: 0.1099 - mae: 0.1479 - root_mean_squared_error: 0.3315 - mean_squared_logarithmic_error: 0.0404
Epoch 62/100
14/14 [==============================] - 0s 18ms/step - loss: 0.1299 - mse: 0.1299 - mae: 0.1643 - root_mean_squared_error: 0.3605 - mean_squared_logarithmic_error: 0.0454
Epoch 63/100
14/14 [==============================] - 0s 21ms/step - loss: 0.1143 - mse: 0.1143 - mae: 0.1514 - root_mean_squared_error: 0.3381 - mean_squared_logarithmic_error: 0.0417
Epoch 64/100
14/14 [==============================] - 0s 22ms/step - loss: 0.1160 - mse:

Epoch 1/100
14/14 [==============================] - 7s 55ms/step - loss: 0.2614 - mse: 0.2614 - mae: 0.1748 - root_mean_squared_error: 0.5113 - mean_squared_logarithmic_error: 0.0749
Epoch 2/100
14/14 [==============================] - 0s 20ms/step - loss: 0.2456 - mse: 0.2456 - mae: 0.2429 - root_mean_squared_error: 0.4956 - mean_squared_logarithmic_error: 0.0715
Epoch 3/100
14/14 [==============================] - 0s 21ms/step - loss: 0.2396 - mse: 0.2396 - mae: 0.2586 - root_mean_squared_error: 0.4895 - mean_squared_logarithmic_error: 0.0721
Epoch 4/100
14/14 [==============================] - 0s 20ms/step - loss: 0.2327 - mse: 0.2327 - mae: 0.2317 - root_mean_squared_error: 0.4824 - mean_squared_logarithmic_error: 0.0686
Epoch 5/100
14/14 [==============================] - 0s 20ms/step - loss: 0.2221 - mse: 0.2221 - mae: 0.2103 - root_mean_squared_error: 0.4712 - mean_squared_logarithmic_error: 0.0637
Epoch 6/100
14/14 [==============================] - 0s 20ms/step - loss: 0.2135

14/14 [==============================] - 0s 21ms/step - loss: 0.0881 - mse: 0.0881 - mae: 0.1312 - root_mean_squared_error: 0.2968 - mean_squared_logarithmic_error: 0.0351
Epoch 90/100
14/14 [==============================] - 0s 22ms/step - loss: 0.0888 - mse: 0.0888 - mae: 0.1275 - root_mean_squared_error: 0.2980 - mean_squared_logarithmic_error: 0.0358
Epoch 91/100
14/14 [==============================] - 0s 26ms/step - loss: 0.0829 - mse: 0.0829 - mae: 0.1222 - root_mean_squared_error: 0.2879 - mean_squared_logarithmic_error: 0.0332
Epoch 92/100
14/14 [==============================] - 0s 23ms/step - loss: 0.0846 - mse: 0.0846 - mae: 0.1242 - root_mean_squared_error: 0.2909 - mean_squared_logarithmic_error: 0.0344
Epoch 93/100
14/14 [==============================] - 0s 22ms/step - loss: 0.0846 - mse: 0.0846 - mae: 0.1085 - root_mean_squared_error: 0.2909 - mean_squared_logarithmic_error: 0.0335
Epoch 94/100
14/14 [==============================] - 0s 22ms/step - loss: 0.0824 - mse:

In [25]:
%tensorboard --logdir '/home/bavanya/Documents/Training_curves/propv285/'

In [26]:
total_training_time

[43.4110643863678,
 41.74894165992737,
 44.17507529258728,
 46.69575500488281,
 42.1430401802063,
 41.852657079696655,
 42.32924485206604,
 42.12708044052124,
 42.137452840805054,
 43.697152853012085]

In [27]:
print("avg training time: " + str(sum(total_training_time)/30))

avg training time: 14.343915486335755


In [28]:
total_training_loss_history

[{'loss': [0.2639703154563904,
   0.24954131245613098,
   0.24337314069271088,
   0.23697303235530853,
   0.22917059063911438,
   0.22456613183021545,
   0.2151903212070465,
   0.2058430016040802,
   0.19893519580364227,
   0.19271475076675415,
   0.18501071631908417,
   0.1829724907875061,
   0.17899039387702942,
   0.17276445031166077,
   0.16845887899398804,
   0.16694961488246918,
   0.16353929042816162,
   0.15621300041675568,
   0.15277600288391113,
   0.15728923678398132,
   0.15328647196292877,
   0.15071342885494232,
   0.14325174689292908,
   0.1477719247341156,
   0.15719236433506012,
   0.14915767312049866,
   0.13975457847118378,
   0.14429013431072235,
   0.137431800365448,
   0.13648150861263275,
   0.14301861822605133,
   0.1357879340648651,
   0.13615022599697113,
   0.14596135914325714,
   0.1286216527223587,
   0.12594878673553467,
   0.14210665225982666,
   0.13618311285972595,
   0.12656325101852417,
   0.12797923386096954,
   0.12880796194076538,
   0.122561417520

In [29]:
sum_mse = 0
sum_mae = 0
training_mse = []
training_mae = []
for i in range(10):
    sum_mse += float(total_training_loss_history[i]['mse'][-1])
    sum_mae += float(total_training_loss_history[i]['mae'][-1])
    training_mse.append(total_training_loss_history[i]['mse'][-1])
    training_mae.append(total_training_loss_history[i]['mae'][-1])
print("avg mse of training: " + str(sum_mse/30))
print("avg mae of training: " + str(sum_mae/30))

avg mse of training: 0.03063636670509974
avg mae of training: 0.04123102401693662


In [30]:
total_scores

[[0.3619212508201599,
  0.3619212508201599,
  0.18908020853996277,
  0.6015989184379578,
  0.08396267145872116],
 [0.2892734706401825,
  0.2892734706401825,
  0.2026626020669937,
  0.5378414988517761,
  0.07763998210430145],
 [0.35817158222198486,
  0.35817158222198486,
  0.20587122440338135,
  0.598474383354187,
  0.08391427248716354],
 [0.2803228497505188,
  0.2803228497505188,
  0.22754183411598206,
  0.5294552445411682,
  0.0871056616306305],
 [0.3168503940105438,
  0.3168503940105438,
  0.18315774202346802,
  0.5628946423530579,
  0.07624820619821548],
 [0.3582782447338104,
  0.3582782447338104,
  0.22050392627716064,
  0.5985634922981262,
  0.08877206593751907],
 [0.321387380361557,
  0.321387380361557,
  0.1948113739490509,
  0.5669103860855103,
  0.0783807784318924],
 [0.295612633228302,
  0.295612633228302,
  0.19407512247562408,
  0.5437027215957642,
  0.07726459205150604],
 [0.32670173048973083,
  0.32670173048973083,
  0.23101921379566193,
  0.5715782642364502,
  0.08660048

In [1]:
a = [[0.3619212508201599,
  0.3619212508201599,
  0.18908020853996277,
  0.6015989184379578,
  0.08396267145872116],
 [0.2892734706401825,
  0.2892734706401825,
  0.2026626020669937,
  0.5378414988517761,
  0.07763998210430145],
 [0.35817158222198486,
  0.35817158222198486,
  0.20587122440338135,
  0.598474383354187,
  0.08391427248716354],
 [0.2803228497505188,
  0.2803228497505188,
  0.22754183411598206,
  0.5294552445411682,
  0.0871056616306305],
 [0.3168503940105438,
  0.3168503940105438,
  0.18315774202346802,
  0.5628946423530579,
  0.07624820619821548],
 [0.3582782447338104,
  0.3582782447338104,
  0.22050392627716064,
  0.5985634922981262,
  0.08877206593751907],
 [0.321387380361557,
  0.321387380361557,
  0.1948113739490509,
  0.5669103860855103,
  0.0783807784318924],
 [0.295612633228302,
  0.295612633228302,
  0.19407512247562408,
  0.5437027215957642,
  0.07726459205150604],
 [0.32670173048973083,
  0.32670173048973083,
  0.23101921379566193,
  0.5715782642364502,
  0.08660048246383667],
 [0.36798322200775146,
  0.36798322200775146,
  0.21455925703048706,
  0.606616199016571,
  0.09024269878864288]]

In [2]:
test_mae = []
for i in range(10):
    test_mae.append(a[i][2])

In [4]:
test_mae

[0.18908020853996277,
 0.2026626020669937,
 0.20587122440338135,
 0.22754183411598206,
 0.18315774202346802,
 0.22050392627716064,
 0.1948113739490509,
 0.19407512247562408,
 0.23101921379566193,
 0.21455925703048706]

In [34]:
sum_mse = 0
sum_mae = 0
sum_are = 0
test_mse = []
test_mae = []
for i in range(10):
    sum_mse += float(total_scores[i][0])
    sum_mae += float(total_scores[i][2])
    test_mse.append(total_scores[i][0])
    test_mae.append(total_scores[i][2])
    sum_are += float(test_ARE[i])
print("avg mse: " + str(sum_mse/30))
print("avg mae: " + str(sum_mae/30))
print("avg are: " + str(sum_are/30))

avg mse: 0.10921675860881805
avg mae: 0.06877608348925908
avg are: 0.03342932090163231


In [35]:
print("median training mse: " + str(statistics.median(training_mse)))
print("median testing mse: " + str(statistics.median(test_mse)))

median training mse: 0.09036345407366753
median testing mse: 0.3240445554256439


In [36]:
print("median training mae: " + str(statistics.median(training_mae)))
print("median testing mae: " + str(statistics.median(test_mae)))

median training mae: 0.12378169968724251
median testing mae: 0.20426691323518753


In [37]:
print("median testing ARE: " + str(statistics.median(test_ARE)))

median testing ARE: 0.10184014588594437


In [38]:
test_ARE

[0.10619469,
 0.09495716,
 0.10373648,
 0.09263942,
 0.10085686,
 0.105773285,
 0.10282343,
 0.1043686,
 0.09790701,
 0.0936227]